This notebook aims to use simple linear models to predict the returns of a specific stock given historical data.

## Get initial data

In [68]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [69]:
GetFacebookInformation = yf.Ticker("META")


# put the data into a pandas dataframe
df = GetFacebookInformation.history(period="1mo")
# plot the data
fig = go.Figure(go.Scatter(x=df.index, y=df['Close'], mode='markers', name='Close'))
fig.update_layout(
    title='Facebook Stock Price',
    xaxis_title='Date',
    yaxis_title='Closing Price (USD)',
    xaxis_rangeslider_visible=False
)

fig.show()



## Perform linear regression

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# splitting data into training and testing sets (80-20 split)
X = df.index.astype('int64').values.reshape(-1, 1)
y = df['Close'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")


Coefficients: [-5.04951204e-14]
Intercept: 88555.20034229648
Mean Squared Error (MSE): 213.71001747256582
R² Score: 0.7044915641791174


In [71]:
# write in the slope and intercept form
Y = model.coef_ * X + model.intercept_
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.flatten(), y=y, mode='markers', name='Actual Data', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=X.flatten(), y=Y.flatten(), mode='lines', name='Regression Line', line=dict(color='red')))
fig.update_layout(
    title='Linear Regression',
    xaxis_title='X',
    yaxis_title='y',
    legend_title='Legend'
)
fig.show()

In [72]:
# get date for next day
last_date = df.index[-1]
next_date = last_date + dt.timedelta(days=1)
next_date = next_date.strftime('%Y-%m-%d')

print(f"Next Date: {next_date}")

Next Date: 2025-03-15
